# Database Preprocessing and Cleaning
<hr>
Now that we got the data somewhere that it is easily accessible, now we just need to clean it and get it to a nice workable state. Once we accomplish that, then we'll be able to lock it down and make sure it can't be overwritten.

In [3]:
import pandas as pd
from sqlalchemy import create_engine
from pandas.io import sql

# Database creds, don't share on public repo
hostname='95.217.156.58'
dbname='crimeStats'
uname='sandwich'
pwd='321#@!IdiotSandwich'
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))


# get database as df
maindf = pd.read_sql('SELECT * FROM crimeTableRaw', con=engine)

In [70]:
df = df.reset_index()
del df['index']

In [71]:
df.isna().sum()

level_0                            0
arrest_key                         0
arrest_date                        0
pd_cd                            270
pd_desc                         8950
ky_cd                           8961
ofns_desc                       8943
law_code                         207
law_cat_cd                     17391
arrest_boro                        9
arrest_precinct                    0
jurisdiction_code                  9
age_group                         16
perp_sex                           0
perp_race                          0
x_coord_cd                         1
y_coord_cd                         2
latitude                          22
longitude                          6
:@computed_region_efsh_h5xi    23792
:@computed_region_f5dn_yrer     7769
:@computed_region_yeji_bk3q     7821
:@computed_region_92fq_4b7q     7816
:@computed_region_sbqj_enih     7965
dtype: int64

With the amount of nulls we have, we should establish which values we absolutely need based on the needs of our project. Since we would like to map crime and look at crime historically, "latitude", "longitude", and "arrest_date" are non negotiable. Thankfully, at most, we should only lose 28 rows at most.

In [72]:
df = df[df['latitude'].notna()]

In [73]:
df = df[df['longitude'].notna()]

while age range isn't something we initially considered, since there are only 16 null values out of over 5 million, it could be valuable to drop those rows so we have that as a cleanly populated atttibute.

In [74]:
df = df[df['age_group'].notna()]

While looking at a sample of the data, I noticed that there was a value of '944' which doesn't seem like a age group. So, we decided that it'd be best to get a list of all the unique age groups.

In [ ]:
# all the age groups
unqiue_values=pd.unique(df['age_group'])
len(unqiue_values)

That's not bad. Only 136 values that aren't the right age group. We'll just drop them.

In [ ]:
#converting the result into lists
list(unqiue_values)

#the age groups we want to keep
imp = ['45-64', '25-44', '18-24', '<18', '65+']

#this will create a list of age groups we don't want to keep
useless = [x for x in unqiue_values if x not in imp]

#removes those age groups
df=df.loc[~df.age_group.isin(useless)]

The next thing that stands out is x and y coordinates. I am curious if these may be redundant because we already have lat and long. 

After looking into it, it seems the "x_coord_cd" and "y_coord_cd" are attributes of the State Plane Coordinate System, which is an almost completely outdated way to lookup locations(<a href="https://www.usgs.gov/faqs/what-state-plane-coordinate-system-can-gps-provide-coordinates-these-values?qt-news_science_products=0#qt-news_science_products">see here</a>). Since that is the case we can remove those two rows completely.

In [75]:
del df['x_coord_cd']
del df['y_coord_cd']

Now Looking at all those "@" columns that have a bunch of null values.
 - computed_region_efsh_h5xi = Zip Code
 - computed_region_f5dn_yrer = Community District
 - computed_region_yeji_bk3q = Borough Boundaries
 - computed_region_92fq_4b7q = City Council Districts
 - computed_region_sbqj_enih = Police Precincts
 
What should we do with all these values? Since they provide actionable information (especially the zip code), we should probably keep them. 

In fact, we may actually be able to recover some information using geocoding libraries. We can see if we can grab some zip codes. The rest, we'll leave null, but keep on hand for specialized use if we decide they're useful for something since their nulls won't impede our main interests.

***update:*** upon investigation, these don't appear to be zip codes where the crime occurs. This means using the lat and long to provide the remaining zip codes wouldn't be right. Moreso, there are a lot of values like "1" and "26001", that aren't valid zip codes. This leads me to believe that this column is no use to us, so we'll get rid of it.

In [76]:
del df[':@computed_region_efsh_h5xi']

In [77]:
df

,level_0,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,...,jurisdiction_code,age_group,perp_sex,perp_race,latitude,longitude,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih
1,1,192799737,2019-01-26T00:00:00.000,177,SEXUAL ABUSE,116,SEX CRIMES,PL 1306503,F,M,...,0,45-64,M,BLACK,40.800694,-73.94110928599997,7.0,4,36.0,16
2,2,193260691,2019-02-06T00:00:00.000,None,None,None,None,PL 2203400,F,M,...,0,25-44,M,UNKNOWN,40.757839,-73.99121211099998,12.0,4,10.0,8
3,3,149117452,2016-01-06T00:00:00.000,153,RAPE 3,104,RAPE,PL 1302503,F,K,...,0,25-44,M,BLACK,40.648650,-73.95033556299995,61.0,2,11.0,40
4,4,190049060,2018-11-15T00:00:00.000,157,RAPE 1,104,RAPE,PL 1303501,F,K,...,0,25-44,M,BLACK,40.674583,-73.93022154099998,16.0,2,49.0,49
5,5,24288194,2006-09-13T00:00:00.000,203,"TRESPASS 3, CRIMINAL",352,CRIMINAL TRESPASS,PL 140100E,M,K,...,2,45-64,M,BLACK,40.671254,-73.926713851,16.0,2,49.0,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5011995,5011995,24068627,2006-08-28T00:00:00.000,849,"NY STATE LAWS,UNCLASSIFIED VIOLATION",677,OTHER STATE LAWS,LOC000000V,V,Q,...,0,25-44,M,WHITE,40.747590,-73.88439699899999,65.0,3,21.0,73
5011996,5011996,24566897,2006-09-30T00:00:00.000,705,"FORGERY,ETC.-MISD.",358,OFFENSES INVOLVING FRAUD,PL 1657100,M,Q,...,0,25-44,M,BLACK HISPANIC,40.748281,-73.87788142999993,65.0,3,21.0,73
5011997,5011997,24578122,2006-10-01T00:00:00.000,904,"INTOXICATED DRIVING,ALCOHOL",119,INTOXICATED/IMPAIRED DRIVING,VTL11920F2,F,Q,...,0,25-44,M,WHITE HISPANIC,40.752037,-73.86907874399995,65.0,3,21.0,73
5011998,5011998,24172707,2006-09-06T00:00:00.000,503,"CONTROLLED SUBSTANCE,INTENT TO SELL 3",117,DANGEROUS DRUGS,PL 2201601,F,K,...,0,45-64,M,BLACK,40.650052,-73.964169327,60.0,2,11.0,43


Another two attributes that don't have a lot of null values which make it worth dropping the NA values are ARREST_BORO and JURISDICTION_CODE. We could figure out the arrest boro based on the lat and long of the arrest, but with there only being 9/5 million null rows, it's easier to just delete them and move on.

In [78]:
df = df[df['arrest_boro'].notna()]
df = df[df['jurisdiction_code'].notna()]

In [79]:
# pd_cd and ky_cd can be deleted because they're internal use codes

del df['pd_cd']
del df['ky_cd']

In [80]:
# blanks law codes means we have no idea what was committed
# ofns_desc is also blank, so these 200 values don't do much for us

df[df['law_code'].isna()]

,level_0,arrest_key,arrest_date,pd_desc,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,jurisdiction_code,age_group,perp_sex,perp_race,latitude,longitude,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih
28,28,188086111,2018-09-27T00:00:00.000,None,None,None,None,B,50,0,25-44,M,WHITE HISPANIC,40.910270,-73.89711662799994,48.0,5,40.0,33
61,61,188086113,2018-09-27T00:00:00.000,None,None,None,None,B,50,0,25-44,M,WHITE HISPANIC,40.910270,-73.89711662799994,48.0,5,40.0,33
73,73,188785833,2018-10-15T00:00:00.000,None,None,None,None,K,67,0,25-44,M,BLACK,40.651465,-73.95423641599997,61.0,2,11.0,40
99,99,188304699,2018-10-03T00:00:00.000,None,None,None,None,B,47,0,25-44,M,WHITE HISPANIC,40.872162,-73.86614123499999,29.0,5,22.0,30
307,307,190932018,2018-12-09T00:00:00.000,None,None,None,None,Q,115,0,18-24,M,WHITE HISPANIC,40.751099,-73.86064222299996,65.0,3,21.0,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5002571,5002571,24036545,2006-08-27T00:00:00.000,None,None,None,None,K,75,0,<18,M,WHITE HISPANIC,40.675194,-73.87342954799993,45.0,2,25.0,47
5004838,5004838,24028439,2006-08-25T00:00:00.000,None,None,None,None,M,14,3,25-44,M,BLACK,40.753939,-73.99484365499995,12.0,4,10.0,8
5005185,5005185,24028439,2006-08-25T00:00:00.000,None,None,None,None,M,14,3,25-44,M,BLACK,40.753939,-73.99484365499995,12.0,4,10.0,8
5005370,5005370,24289053,2006-09-13T00:00:00.000,None,None,None,None,Q,113,0,18-24,M,BLACK,40.673395,-73.775671989,41.0,3,46.0,71


In [81]:
df = df[df['law_code'].notna()]

Now lets see if we can get more law info from other columns by using law_code to see if we can recover the descriptions

In [82]:
result = pd.concat([df[df['ofns_desc'].isna()]["law_code"], df[df['ofns_desc'].isna()]["arrest_key"]], axis=1)
result

,law_code,arrest_key
2,PL 2203400,193260691
12,PL 1651601,189176315
20,PL 1651601,189974181
34,PL 1651601,186712976
54,PL 2203400,191943063
...,...,...
4803094,FCA000000M,27155306
4866389,MLR00000AM,26139862
4874249,FCA000000M,26334227
4875072,FCA000000M,26334227


In [83]:
laws = []
for i in result["law_code"]:
    if i not in laws:
        laws.append(i)

In [84]:
# check if law codes and full df have a diff
# amount of missing data

df[df['law_code'].isin(laws)].isna().sum()["ofns_desc"]

8736

In [85]:
df.isna().sum()["ofns_desc"]

8736

thats not promising... theres almost no difference. So maybe we can take another approach if the count of unique missing laws aren't too large.

By collecting NY penal code, I think we'll be able to recover a fair amount of decriptions

In [86]:
print(len(laws))

65


In [87]:
# 65 isnt too bad, especially when it'll recover us a lot of records
# so i collevted them on a website (only PL ones)

lawCodes = pd.read_csv("missingLaws.csv",encoding = "ISO-8859-1")

In [88]:
# apply the found law code descriptions to the dataframe

for index, row in lawCodes.iterrows():
    df.loc[(df.law_code == row["law_code"]),'ofns_desc']=row["general crime"]
    df.loc[(df.law_code == row["law_code"]),'pd_desc']=row["specific crime"]


In [89]:
df.isna().sum()["ofns_desc"]

127

We recovered over 8500, so I'll call that a win. I think we can drop the remainders since the external law code itself won't provide a lot of insight

In [90]:
df = df[df['pd_desc'].notna()]
df = df[df['ofns_desc'].notna()]

In [91]:
df

,level_0,arrest_key,arrest_date,pd_desc,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,jurisdiction_code,age_group,perp_sex,perp_race,latitude,longitude,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih
1,1,192799737,2019-01-26T00:00:00.000,SEXUAL ABUSE,SEX CRIMES,PL 1306503,F,M,25,0,45-64,M,BLACK,40.800694,-73.94110928599997,7.0,4,36.0,16
2,2,193260691,2019-02-06T00:00:00.000,CRIMINAL SALE OF A CONTROLLED SUBSTANCE,CONTROLLED SUBSTANCES OFFENSES,PL 2203400,F,M,14,0,25-44,M,UNKNOWN,40.757839,-73.99121211099998,12.0,4,10.0,8
3,3,149117452,2016-01-06T00:00:00.000,RAPE 3,RAPE,PL 1302503,F,K,67,0,25-44,M,BLACK,40.648650,-73.95033556299995,61.0,2,11.0,40
4,4,190049060,2018-11-15T00:00:00.000,RAPE 1,RAPE,PL 1303501,F,K,77,0,25-44,M,BLACK,40.674583,-73.93022154099998,16.0,2,49.0,49
5,5,24288194,2006-09-13T00:00:00.000,"TRESPASS 3, CRIMINAL",CRIMINAL TRESPASS,PL 140100E,M,K,77,2,45-64,M,BLACK,40.671254,-73.926713851,16.0,2,49.0,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5011995,5011995,24068627,2006-08-28T00:00:00.000,"NY STATE LAWS,UNCLASSIFIED VIOLATION",OTHER STATE LAWS,LOC000000V,V,Q,115,0,25-44,M,WHITE,40.747590,-73.88439699899999,65.0,3,21.0,73
5011996,5011996,24566897,2006-09-30T00:00:00.000,"FORGERY,ETC.-MISD.",OFFENSES INVOLVING FRAUD,PL 1657100,M,Q,115,0,25-44,M,BLACK HISPANIC,40.748281,-73.87788142999993,65.0,3,21.0,73
5011997,5011997,24578122,2006-10-01T00:00:00.000,"INTOXICATED DRIVING,ALCOHOL",INTOXICATED/IMPAIRED DRIVING,VTL11920F2,F,Q,115,0,25-44,M,WHITE HISPANIC,40.752037,-73.86907874399995,65.0,3,21.0,73
5011998,5011998,24172707,2006-09-06T00:00:00.000,"CONTROLLED SUBSTANCE,INTENT TO SELL 3",DANGEROUS DRUGS,PL 2201601,F,K,70,0,45-64,M,BLACK,40.650052,-73.964169327,60.0,2,11.0,43


In [98]:
# now lets clean up the date

arrDates = df['arrest_date'].str.slice_replace(10, 23, '')

In [99]:
final = df
del final["arrest_date"]
final = pd.concat([final, arrDates], axis=1)

So law_cat_cd still has a fair amount of null values, but I think we can keep that frame as is, and if we want to evaluate the severity of crimes or measure density of "harsher" crimes we can do so later by dropping those nulls. The data is now in a very usable state, and I think its ready to be saved... After we delete "level_0" and reorder cols.

In [101]:
del final["level_0"]

In [104]:
cols = final.columns.tolist()
cols

['arrest_key',
 'pd_desc',
 'ofns_desc',
 'law_code',
 'law_cat_cd',
 'arrest_boro',
 'arrest_precinct',
 'jurisdiction_code',
 'age_group',
 'perp_sex',
 'perp_race',
 'latitude',
 'longitude',
 ':@computed_region_f5dn_yrer',
 ':@computed_region_yeji_bk3q',
 ':@computed_region_92fq_4b7q',
 ':@computed_region_sbqj_enih',
 'arrest_date']

In [105]:
# this looks like a cleaner order
cols = ['arrest_key',
        'arrest_date',
        'pd_desc',
        'ofns_desc',
        'law_code',
        'law_cat_cd',
        'age_group',
        'perp_sex',
        'perp_race',
        'latitude',
        'longitude',
        'arrest_boro',
        'arrest_precinct',
        'jurisdiction_code',
        ':@computed_region_f5dn_yrer',
        ':@computed_region_yeji_bk3q',
        ':@computed_region_92fq_4b7q',
        ':@computed_region_sbqj_enih']

In [106]:
final = final[cols]

In [110]:
# one last thing I almost forgot, checking/dropping dupes

final = final.drop_duplicates()

In [112]:
# Now we can finally upload our clean data and get started on EDA

final.to_sql('crimeTable', engine, if_exists='replace', index=False)

In [4]:
lawCodes = pd.read_csv("missingLaws.csv",encoding = "ISO-8859-1")

def batchProcess(df, lawCodes):
    df = df.reset_index()
    del df['index']
    df = df[df['latitude'].notna()]
    df = df[df['longitude'].notna()]
    df = df[df['age_group'].notna()]
    del df['x_coord_cd']
    del df['y_coord_cd']
    del df[':@computed_region_efsh_h5xi']
    df = df[df['arrest_boro'].notna()]
    df = df[df['jurisdiction_code'].notna()]
    del df['pd_cd']
    del df['ky_cd']
    df = df[df['law_code'].notna()]
    result = pd.concat([df[df['ofns_desc'].isna()]["law_code"], df[df['ofns_desc'].isna()]["arrest_key"]], axis=1)
    for index, row in lawCodes.iterrows():
        df.loc[(df.law_code == row["law_code"]),'ofns_desc']=row["general crime"]
        df.loc[(df.law_code == row["law_code"]),'pd_desc']=row["specific crime"]
    df = df[df['pd_desc'].notna()]
    df = df[df['ofns_desc'].notna()]
    arrDates = df['arrest_date'].str.slice_replace(10, 23, '')
    final = df
    del final["arrest_date"]
    final = pd.concat([final, arrDates], axis=1)
    del final["level_0"]
    cols = ['arrest_key',
            'arrest_date',
            'pd_desc',
            'ofns_desc',
            'law_code',
            'law_cat_cd',
            'age_group',
            'perp_sex',
            'perp_race',
            'latitude',
            'longitude',
            'arrest_boro',
            'arrest_precinct',
            'jurisdiction_code',
            ':@computed_region_f5dn_yrer',
            ':@computed_region_yeji_bk3q',
            ':@computed_region_92fq_4b7q',
            ':@computed_region_sbqj_enih']
    final = final[cols]
    final = final.drop_duplicates()
    final.to_sql('crimeTableSegmented', engine, if_exists='append', index=False)

In [8]:
maindf[4000:8000]

,index,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,...,perp_race,x_coord_cd,y_coord_cd,latitude,longitude,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih
4000,0,193078640,2019-02-02T00:00:00.000,922.0,"TRAFFIC,UNCLASSIFIED MISDEMEAN",348.0,VEHICLE AND TRAFFIC LAWS,VTL05110A4,M,Q,...,ASIAN / PACIFIC ISLANDER,1034704.0,201410.0,40.719365,-73.817988,14509.0,25.0,3.0,24.0,65.0
4001,1,192578182,2019-01-21T00:00:00.000,114.0,OBSTR BREATH/CIRCUL,344.0,ASSAULT 3 & RELATED OFFENSES,PL 121110A,M,B,...,WHITE HISPANIC,1020649.0,254200.0,40.864328,-73.868406,11605.0,59.0,5.0,22.0,32.0
4002,2,194277979,2019-03-05T00:00:00.000,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200000,M,B,...,BLACK HISPANIC,1019141.0,259102.0,40.877789,-73.873832,11605.0,24.0,5.0,40.0,34.0
4003,3,195134213,2019-03-25T00:00:00.000,203.0,"TRESPASS 3, CRIMINAL",352.0,CRIMINAL TRESPASS,PL 140100A,M,K,...,ASIAN / PACIFIC ISLANDER,998150.0,154850.0,40.591701,-73.949953,13826.0,32.0,2.0,15.0,36.0
4004,4,193722020,2019-02-18T00:00:00.000,750.0,RESISTING ARREST,359.0,OFFENSES AGAINST PUBLIC ADMINI,PL 2053000,M,Q,...,BLACK,1039811.0,192297.0,40.694321,-73.799640,24668.0,41.0,3.0,46.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,995,193115194,2019-02-03T00:00:00.000,969.0,"TRAFFIC,UNCLASSIFIED INFRACTIO",881.0,OTHER TRAFFIC INFRACTION,VTL051101A,M,Q,...,BLACK HISPANIC,1029166.0,151271.0,40.581774,-73.838301,20532.0,51.0,3.0,41.0,58.0
7996,996,198692066,2019-06-19T00:00:00.000,511.0,"CONTROLLED SUBSTANCE, POSSESSI",235.0,DANGEROUS DRUGS,PL 2200300,M,Q,...,WHITE HISPANIC,996424.0,210017.0,40.743125,-73.956068,16858.0,53.0,3.0,33.0,66.0
7997,997,192792423,2019-01-26T00:00:00.000,782.0,"WEAPONS, POSSESSION, ETC",236.0,DANGEROUS WEAPONS,PL 2650101,M,B,...,BLACK,1022144.0,263350.0,40.889436,-73.862949,11275.0,29.0,5.0,2.0,30.0
7998,998,201814849,2019-08-31T00:00:00.000,707.0,"IMPERSONATION 2, PUBLIC SERVAN",340.0,FRAUDS,PL 1902300,M,Q,...,BLACK,1041879.0,197084.0,40.707447,-73.792142,24340.0,41.0,3.0,6.0,61.0


In [9]:
# for some reason the code messes up column wise with large operations, so chunking the code fixes that issue
a = 0
b = 4000
for i in range(1253):
    percent = float(i) / float(1253) * 100
    print("progress: " + str(percent) + "%")
    batchProcess(maindf[a:b], lawCodes)
    a = a+4000
    b = b+4000

progress: 0.0%
progress: 0.07980845969672785%
progress: 0.1596169193934557%
progress: 0.23942537909018355%
progress: 0.3192338387869114%
progress: 0.39904229848363926%
progress: 0.4788507581803671%
progress: 0.5586592178770949%
progress: 0.6384676775738228%
progress: 0.7182761372705506%
progress: 0.7980845969672785%
progress: 0.8778930566640064%
progress: 0.9577015163607342%
progress: 1.0375099760574622%
progress: 1.1173184357541899%
progress: 1.1971268954509178%
progress: 1.2769353551476457%
progress: 1.3567438148443736%
progress: 1.4365522745411012%
progress: 1.5163607342378291%
progress: 1.596169193934557%
progress: 1.675977653631285%
progress: 1.7557861133280128%
progress: 1.8355945730247407%
progress: 1.9154030327214684%
progress: 1.9952114924181963%
progress: 2.0750199521149244%
progress: 2.154828411811652%
progress: 2.2346368715083798%
progress: 2.314445331205108%
progress: 2.3942537909018355%
progress: 2.4740622505985637%
progress: 2.5538707102952913%
progress: 2.63367916999201

progress: 22.266560255387073%
progress: 22.3463687150838%
progress: 22.426177174780527%
progress: 22.505985634477256%
progress: 22.585794094173984%
progress: 22.66560255387071%
progress: 22.745411013567438%
progress: 22.825219473264166%
progress: 22.905027932960895%
progress: 22.98483639265762%
progress: 23.06464485235435%
progress: 23.144453312051077%
progress: 23.224261771747805%
progress: 23.30407023144453%
progress: 23.38387869114126%
progress: 23.463687150837988%
progress: 23.543495610534716%
progress: 23.623304070231445%
progress: 23.703112529928173%
progress: 23.782920989624902%
progress: 23.86272944932163%
progress: 23.942537909018355%
progress: 24.022346368715084%
progress: 24.102154828411813%
progress: 24.18196328810854%
progress: 24.261771747805266%
progress: 24.341580207501995%
progress: 24.421388667198723%
progress: 24.501197126895452%
progress: 24.581005586592177%
progress: 24.660814046288905%
progress: 24.740622505985634%
progress: 24.820430965682363%
progress: 24.900239

progress: 44.53312051077415%
progress: 44.612928970470875%
progress: 44.6927374301676%
progress: 44.772545889864325%
progress: 44.852354349561054%
progress: 44.93216280925778%
progress: 45.01197126895451%
progress: 45.09177972865124%
progress: 45.17158818834797%
progress: 45.2513966480447%
progress: 45.33120510774142%
progress: 45.41101356743815%
progress: 45.490822027134875%
progress: 45.570630486831604%
progress: 45.65043894652833%
progress: 45.73024740622506%
progress: 45.81005586592179%
progress: 45.88986432561852%
progress: 45.96967278531524%
progress: 46.04948124501197%
progress: 46.1292897047087%
progress: 46.209098164405425%
progress: 46.288906624102154%
progress: 46.36871508379888%
progress: 46.44852354349561%
progress: 46.52833200319234%
progress: 46.60814046288906%
progress: 46.68794892258579%
progress: 46.76775738228252%
progress: 46.84756584197925%
progress: 46.927374301675975%
progress: 47.007182761372704%
progress: 47.08699122106943%
progress: 47.16679968076616%
progress

progress: 66.95929768555466%
progress: 67.0391061452514%
progress: 67.11891460494812%
progress: 67.19872306464485%
progress: 67.27853152434157%
progress: 67.35833998403831%
progress: 67.43814844373503%
progress: 67.51795690343177%
progress: 67.59776536312849%
progress: 67.67757382282522%
progress: 67.75738228252195%
progress: 67.83719074221868%
progress: 67.9169992019154%
progress: 67.99680766161214%
progress: 68.07661612130886%
progress: 68.15642458100558%
progress: 68.23623304070232%
progress: 68.31604150039904%
progress: 68.39584996009577%
progress: 68.4756584197925%
progress: 68.55546687948923%
progress: 68.63527533918595%
progress: 68.71508379888269%
progress: 68.79489225857941%
progress: 68.87470071827615%
progress: 68.95450917797287%
progress: 69.0343176376696%
progress: 69.11412609736632%
progress: 69.19393455706306%
progress: 69.27374301675978%
progress: 69.35355147645652%
progress: 69.43335993615322%
progress: 69.51316839584996%
progress: 69.59297685554668%
progress: 69.67278

progress: 89.70470869912211%
progress: 89.78451715881883%
progress: 89.86432561851556%
progress: 89.94413407821229%
progress: 90.02394253790902%
progress: 90.10375099760574%
progress: 90.18355945730248%
progress: 90.2633679169992%
progress: 90.34317637669594%
progress: 90.42298483639266%
progress: 90.5027932960894%
progress: 90.58260175578611%
progress: 90.66241021548284%
progress: 90.74221867517956%
progress: 90.8220271348763%
progress: 90.90183559457301%
progress: 90.98164405426975%
progress: 91.06145251396647%
progress: 91.14126097366321%
progress: 91.22106943335993%
progress: 91.30087789305666%
progress: 91.38068635275339%
progress: 91.46049481245012%
progress: 91.54030327214684%
progress: 91.62011173184358%
progress: 91.6999201915403%
progress: 91.77972865123704%
progress: 91.85953711093376%
progress: 91.93934557063048%
progress: 92.01915403032721%
progress: 92.09896249002394%
progress: 92.17877094972067%
progress: 92.2585794094174%
progress: 92.33838786911413%
progress: 92.418196

In [10]:
# do a final drop duplicates across the entire dataset, and reupload to crimeTable
segmented = pd.read_sql('SELECT * FROM crimeTableSegmented', con=engine)
segmented = segmented.drop_duplicates()

segmented.to_sql('crimeTable', engine, if_exists='replace', index=False)